# RAG example

Train encoder (if you don't have any checkpoint)

In [ ]:
!python /content/Altegrad-project-MVA/generative/sft_llm_mapper/training/train_encoder.py \
--data_dir /content/drive/MyDrive/molecule-captioning/data/ \
--train_emb /content/drive/MyDrive/molecule-captioning/embeddings/train_embeddings_sentence-transformers_all-mpnet-base-v2.csv \
--val_emb /content/drive/MyDrive/molecule-captioning/embeddings/validation_embeddings_sentence-transformers_all-mpnet-base-v2.csv \
--out_ckpt /content/drive/MyDrive/molecule-captioning/checkpoints/encoder_for_sft.pt \
--epochs 10

Perform the alignement

In [ ]:
!python /content/Altegrad-project-MVA/generative/sft_llm_mapper/training/alignement.py \
--llm biogpt \
--train_data /content/drive/MyDrive/molecule-captioning/data/train_graphs.pkl \
--val_data /content/drive/MyDrive/molecule-captioning/data/validation_graphs.pkl \
--graph_ckpt /content/drive/MyDrive/molecule-captioning/checkpoints/encoder_for_sft.pt \
--epochs 10

Perform the Supervised Fine tuning

In [ ]:
!python /content/Altegrad-project-MVA/generative/sft_llm_mapper/training/sft.py \
--train_data /content/drive/MyDrive/molecule-captioning/data/train_graphs.pkl \
--val_data /content/drive/MyDrive/molecule-captioning/data/validation_graphs.pkl \
--encoder_ckpt /content/drive/MyDrive/molecule-captioning/checkpoints/encoder_for_sft.pt \
--mapper_ckpt /content/stage1_mapper.pt \
--llm biogpt \
--epochs 15

Build the FAISS index

In [ ]:
!python /content/Altegrad-project-MVA/generative/sft_llm_mapper/inference/build_graph_faiss.py \
  --train_data /content/drive/MyDrive/molecule-captioning/data/train_graphs.pkl \
  --encoder_ckpt /content/drive/MyDrive/molecule-captioning/checkpoints/encoder_for_sft.pt \
  --out_index rag/graph_index.faiss \
  --device cuda

Inference using RAG

!python /content/Altegrad-project-MVA/generative/sft_llm_mapper/inference/inference_rag.py \
--test_data /content/drive/MyDrive/molecule-captioning/data/test_graphs.pkl \
--encoder_ckpt /content/drive/MyDrive/molecule-captioning/checkpoints/encoder_for_sft.pt \
--lora_path /content/checkpoints/stage2/lora \
--faiss_index rag/graph_index.faiss \
--faiss_texts rag/graph_index.faiss.texts.pt \
--mapper_ckpt /content/stage1_mapper.pt \
--llm biogpt 